In [33]:
%autoreload 2

In [42]:
from collections import defaultdict
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import typing
import sklearn
from sklearn.model_selection import train_test_split


In [25]:
FITNESS_DATA_FILE = '../data/fitness_scores.csv'
NON_FEATURE_COLUMNS = set(['Index', 'src_file', 'game_name', 'domain_name', 'real', 'original_game_name'])


def regrowth_game_name_cleanup(df: pd.DataFrame):
    regrowth_games = df[df.src_file == 'ast-regrwoth-samples.pddl']
    new_game_names = []
    original_game_names = []
    game_name_counter = defaultdict(lambda: 0)
    for i, row in regrowth_games.iterrows():
        game_name = row.game_name
        original_game_names.append(game_name)
        new_game_name = f'{game_name}-{game_name_counter[game_name]}'
        new_game_names.append(new_game_name)
        game_name_counter[game_name] += 1

    regrowth_games = regrowth_games.assign(game_name=new_game_names, original_game_name=original_game_names)

    df[df.src_file == 'ast-regrwoth-samples.pddl'] = regrowth_games
    return df


fitness_df = pd.read_csv(FITNESS_DATA_FILE)
fitness_df = fitness_df.assign(real=fitness_df.src_file == 'interactive-beta.pddl', original_game_name=None)
fitness_df = regrowth_game_name_cleanup(fitness_df)
fitness_df = fitness_df[~(fitness_df.src_file == 'ast-mle-samples.pddl')]
print(fitness_df.columns)
fitness_df.head()

Index(['Index', 'src_file', 'game_name', 'domain_name', 'variables_defined',
       'all_preferences_used', 'setup_objects_used', 'no_adjacent_once',
       'starts_and_ends_once', 'variable_not_repeated', 'no_nested_logicals',
       'pref_forall_correct', 'real', 'original_game_name'],
      dtype='object')


,Index,src_file,game_name,domain_name,variables_defined,all_preferences_used,setup_objects_used,no_adjacent_once,starts_and_ends_once,variable_not_repeated,no_nested_logicals,pref_forall_correct,real,original_game_name
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,1.0,1.0,1.00,1.0,0.5,1.0,1.0,1.0,True,None
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,1.0,1.0,1.00,1.0,1.0,1.0,1.0,1.0,True,None
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,1.0,1.0,0.00,0.0,0.0,1.0,0.0,0.0,True,None
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,1.0,1.0,0.00,1.0,1.0,1.0,1.0,1.0,True,None
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,1.0,1.0,0.25,1.0,1.0,1.0,1.0,1.0,True,None


In [17]:
fitness_df.drop('Index', axis=1).groupby('src_file').agg([np.mean, np.std])

variables_defined           all_preferences_used  \
                                       mean       std                 mean   
src_file                                                                     
ast-regrwoth-samples.pddl          0.975223  0.130014             0.883740   
interactive-beta.pddl              1.000000  0.000000             0.988095   

                                    setup_objects_used            \
                                std               mean       std   
src_file                                                           
ast-regrwoth-samples.pddl  0.264247           0.391846  0.398341   
interactive-beta.pddl      0.083762           0.419189  0.407367   

                          no_adjacent_once           starts_and_ends_once  \
                                      mean       std                 mean   
src_file                                                                    
ast-regrwoth-samples.pddl         0.867028  0.333025             0.816273   
interactive-beta.pddl             0.887755  0.317290             0.847789   

                                    variable_not_repeated            \
                                std                  mean       std   
src_file                                                              
ast-regrwoth-samples.pddl  0.366220              0.969922  0.096532   
interactive-beta.pddl      0.347843              1.000000  0.000000   

                          no_nested_logicals           pref_forall_correct  \
                                        mean       std                mean   
src_file                                                                     
ast-regrwoth-samples.pddl           0.980867  0.137035            0.765967   
interactive-beta.pddl               0.979592  0.142119            0.862245   

                                    real       
                                std mean  std  
src_file                                       
ast-regrwoth-samples.pddl  0.385004  0.0  0.0  
interactive-beta.pddl      0.312058  1.0  0.0

## Data splitting approach
Under the NCE-style thing I want to try, I basically want to take the real games and split them train/val/test, and then use the regrowth samples corresponding to each game for negative examples and for the normalization.

In [28]:
RANDOM_SEED = 33
TRAINING_PROP = 0.7
VALIDATION_PROP = 0.1
TEST_PROP = 0.2

real_game_names = fitness_df[fitness_df.real].game_name.unique()

train_game_names, val_and_test_game_names = train_test_split(real_game_names, train_size=TRAINING_PROP, random_state=RANDOM_SEED)
val_game_names, test_game_names = train_test_split(val_and_test_game_names, train_size=VALIDATION_PROP/(VALIDATION_PROP+TEST_PROP), random_state=RANDOM_SEED)

train_df = fitness_df[fitness_df.game_name.isin(train_game_names) | fitness_df.original_game_name.isin(train_game_names)]
val_df = fitness_df[fitness_df.game_name.isin(val_game_names) | fitness_df.original_game_name.isin(val_game_names)]
test_df = fitness_df[fitness_df.game_name.isin(test_game_names) | fitness_df.original_game_name.isin(test_game_names)]

normalization_values = {}
for column in train_df.columns:
    if column not in NON_FEATURE_COLUMNS:
        col_mean = train_df[column].mean()
        col_std = train_df[column].std()
        normalization_values[column] = (col_mean, col_std)
        train_df = train_df.assign(**{column: (train_df[column] - col_mean) / col_std})

print(train_df.groupby('real').mean())

            Index  variables_defined  all_preferences_used  \
real                                                         
False  965.735294          -0.012064             -0.025232   
True    47.514706           0.193020              0.403711   

       setup_objects_used  no_adjacent_once  starts_and_ends_once  \
real                                                                
False           -0.002828         -0.003687             -0.004528   
True             0.045244          0.058995              0.072455   

       variable_not_repeated  no_nested_logicals  pref_forall_correct  
real                                                                   
False              -0.019075       -4.263848e-16            -0.015899  
True                0.305201       -4.298033e-16             0.254378  


# Approach
* In each batch, sample some number of real games, and for each of them, subsample some number of the corrupted games.
* Learn a regressor to the fitness (maybe with a hidden layer?)
* Try different regularization approaches/strengths (L1, L2, both)
* Evaluate on held-out validation set, see that it doesn't collapse

## Loss function
I'm inspired by the way Chris Dyer (in https://arxiv.org/abs/1410.8251) writes down the NCE loss:
$$ \mathcal{L}_{NCE_k}^{MC} = \sum_{(w,c) \in \mathcal{D}} \left( \log p (D = 1 \mid c, w) - \sum_{i=1, \bar{w} \mid q} \log p (D = 0 \mid c, \bar{w}) \right) $$
where: 
* $\mathcal{D}$ is the dataset comprised of pairs $(c, w)$ of context and the correct continuation $w$
* $D$ is the label, where $D = 1$ indicates true data and $D = $ indicates noise
* $q$ is a noise proposal distribution from which to sample $\bar{w}$, the noise foil examples for the current context.

In our case: 
* I think of the context $c$ as some game id, where the correct production $w$ is the true game
* Tha labels $D$ behave as they do above, $D = 1$ for a correct game and $D = 0$ for an incorrect one. 
* Our regrowth sampler is the proposal distribution $q$ (from which we could eventually generate as many samples as we want, but currently I pre-generate some number of samples per game).
* Given that my fitness model produces a single output, which I currently pass through a sigmoid, we can think about it as outputting $P(D = 1 \mid c, w)$, and taking 1 - its output as $P(D = 0 \mid \cdot)$

Thus, the procedure becomes:
1. In each batch, sample some number $B$ of true games.
2. For each of those, sample $k$ correuptions of the game. 
3. Compute the loss for this example, and then average over the minibatch.
4. Take a gradient step in this direction.


In [62]:
def df_to_tensor(df: pd.DataFrame, feature_columns: typing.List[str]):
    return torch.tensor(
        np.stack([
            np.concatenate((
                df.loc[df.game_name == game_name, feature_columns].to_numpy(),
                df.loc[df.original_game_name == game_name, feature_columns].to_numpy()
            ))
            for game_name
            in df[df.original_game_name.isna()].game_name.unique()
        ]),
        dtype=torch.float
    )


In [77]:
class FitnessEenrgyModel(nn.Module):
    def __init__(self, n_features: int):
        super().__init__()
        self.n_features = n_features
        self.fc1 = nn.Linear(self.n_features, 1)
        # TODO: consider a hidden layer
        # TODO: do we want a sigmoid or something else? Or nothing at all? 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x


def nce_fitness_loss(scores: torch.Tensor):
    positive_scores = torch.log(scores[:, 0])
    negative_scores = torch.log(1 - scores[:, 1:]).sum(axis=1)
    return -(positive_scores + negative_scores).mean()


def evaluate_fitness(model: nn.Module, data: torch.Tensor):
    model.eval()
    with torch.no_grad():
        scores = model(data)
        positive_scores = scores[:, 0]
        negative_scores = scores[:, 1:]
        game_average_scores = positive_scores - negative_scores.mean(axis=1)
        return positive_scores.mean(), negative_scores.mean(), game_average_scores.mean()


def train_model(model: nn.Module, train_data: torch.Tensor, val_data: torch.Tensor, 
    n_epochs: int = 100, lr: float = 0.01, weight_decay: float = 0.0, print_interval: int = 10,
    batch_size: int = 8, k: int = 4, device: str = 'cpu', seed: int = 33):

    optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)

    train_dataset = TensorDataset(train_data)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_dataset = TensorDataset(val_data)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    torch.manual_seed(seed)
    
    for epoch in range(n_epochs):
        model.train()
        epoch_train_losses = []
        for batch in train_dataloader:
            X = batch[0]
            optimizer.zero_grad()
            negative_indices = torch.randperm(X.shape[1] - 1)[:k] + 1
            indices = torch.cat((torch.tensor([0]), negative_indices))
            X = X[:, indices].to(device)
            scores = model(X)
            loss = nce_fitness_loss(scores)
            epoch_train_losses.append(loss.item())
            loss.backward()
            optimizer.step()

        epoch_val_losses = []

        model.eval()
        with torch.no_grad():
            for batch in val_dataloader:
                X = batch[0]
                negative_indices = torch.randperm(X.shape[1] - 1)[:k] + 1
                indices = torch.cat((torch.tensor([0]), negative_indices))
                X = X[:, indices].to(device)

                scores = model(X)
                loss = nce_fitness_loss(scores)
                epoch_val_losses.append(loss.item())

        if epoch % print_interval == 0:
            print(f'Epoch {epoch}: train loss {np.mean(epoch_train_losses):.4f} | val loss {np.mean(epoch_val_losses):.4f} | weights {model.fc1.weight.data}')
        
    print(evaluate_fitness(model, train_data))
    print(evaluate_fitness(model, val_data))
    



In [86]:
features = ['variables_defined', 'setup_objects_used', 'no_adjacent_once']

train_tensor = df_to_tensor(train_df, features)
val_tensor = df_to_tensor(val_df, features)

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

for seed in range(5):
    print(f'Seed {seed}')
    torch.manual_seed(seed)

    fitness_model = FitnessEenrgyModel(len(features))
    fitness_model.apply(init_weights)

    print(fitness_model.fc1.weight.data)
    train_model(fitness_model, train_tensor, val_tensor, weight_decay=0.1)

Seed 0
tensor([[-0.4717,  0.3284, -0.0243]])
Epoch 0: train loss 3.5379 | val loss 2.9008 | weights tensor([[-0.4408,  0.2979, -0.0268]])
Epoch 10: train loss 2.6789 | val loss 2.6279 | weights tensor([[-0.0723,  0.1395, -0.0002]])
Epoch 20: train loss 2.5363 | val loss 2.5685 | weights tensor([[0.0573, 0.0660, 0.0206]])
Epoch 30: train loss 2.5081 | val loss 2.5625 | weights tensor([[0.1320, 0.0367, 0.0360]])
Epoch 40: train loss 2.4939 | val loss 2.5565 | weights tensor([[0.1777, 0.0215, 0.0453]])
Epoch 50: train loss 2.4793 | val loss 2.5677 | weights tensor([[0.2158, 0.0175, 0.0486]])
Epoch 60: train loss 2.4825 | val loss 2.5662 | weights tensor([[0.2389, 0.0170, 0.0335]])
Epoch 70: train loss 2.4898 | val loss 2.5658 | weights tensor([[0.2558, 0.0134, 0.0258]])
Epoch 80: train loss 2.4807 | val loss 2.5671 | weights tensor([[0.2686, 0.0186, 0.0315]])
Epoch 90: train loss 2.4907 | val loss 2.5781 | weights tensor([[0.2774, 0.0215, 0.0297]])
(tensor(0.2308), tensor(0.2238), tensor(